# cadCAD Tutorials: The Robot and the Marbles, Networks Addition
In [Part 2](https://github.com/BlockScience/SimCAD-Tutorials/blob/master/demos/robot-marbles-part-2/robot-marbles-part-2.ipynb) we introduced the 'language' in which a system must be described in order for it to be interpretable by cadCAD and some of the basic concepts of the library:
* State Variables
* Timestep
* Policies
* State Update Functions
* Partial State Update Blocks
* Simulation Configuration Parameters

In the previous example, we observed how two robotic arms acting in parallel could result in counterintuitive system level behavior despite the simplicity of the individual robotic arm policies. 
In this notebook we'll introduce the concept of networks. This done by extending from two boxes of marbles to *n* boxes which are the nodes in our network. Furthermore, there are are going to be arms between some of the boxes but not others forming a network where the arms are the edges.

__The robot and the marbles__  
* Picture a set of n boxes (`balls`) with an integer number of marbles in each;  a network of robotic arms is capable of taking a marble from their one of their boxes and dropping it into the other one.
* Each robotic arm in the network only controls 2 boxes and they act by moving a marble from one box to the other.
* Each robotic arm is programmed to take one marble at a time from the box containing the largest number of marbles and drop it in the other box. It repeats that process until the boxes contain an equal number of marbles.
* For the purposes of our analysis of this system, suppose we are only interested in monitoring the number of marbles in only their two boxes.

In [1]:
from cadCAD.engine import ExecutionMode, ExecutionContext, Executor
from cadCAD.configuration import Configuration
from cadCAD.configuration.utils.userDefinedObject import udoPipe, UDO#, udcBroker
import pprint as pp
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
%matplotlib inline

T = 50 #iterations in our simulation
n=3 #number of boxes in our network
m= 2 #for barabasi graph type number of edges is (n-2)*m

G = nx.barabasi_albert_graph(n, m)
pp.pprint(G.edges)
# pp.pprint(g.edges)
k = len(G.edges)

EdgeView([(0, 2), (1, 2)])


In [4]:
import inspect
inspect.getmembers(G)

[('__class__', networkx.classes.graph.Graph),
 ('__contains__',
  <bound method Graph.__contains__ of <networkx.classes.graph.Graph object at 0x10f891320>>),
 ('__delattr__',
  <method-wrapper '__delattr__' of Graph object at 0x10f891320>),
 ('__dict__',
  {'graph_attr_dict_factory': dict,
   'node_dict_factory': dict,
   'node_attr_dict_factory': dict,
   'adjlist_outer_dict_factory': dict,
   'adjlist_inner_dict_factory': dict,
   'edge_attr_dict_factory': dict,
   'graph': {},
   '_node': {0: {}, 1: {}, 2: {}},
   '_adj': {0: {2: {}}, 1: {2: {}}, 2: {0: {}, 1: {}}},
   'nodes': NodeView((0, 1, 2))}),
 ('__dir__', <function Graph.__dir__>),
 ('__doc__',
  '\n    Base class for undirected graphs.\n\n    A Graph stores nodes and edges with optional data, or attributes.\n\n    Graphs hold undirected edges.  Self loops are allowed but multiple\n    (parallel) edges are not.\n\n    Nodes can be arbitrary (hashable) Python objects with optional\n    key/value attributes. By convention `Non

In [18]:
getmembers(G, ismethod)

[('__contains__',
  <bound method Graph.__contains__ of <networkx.classes.graph.Graph object at 0x108694438>>),
 ('__getitem__',
  <bound method Graph.__getitem__ of <networkx.classes.graph.Graph object at 0x108694438>>),
 ('__init__',
  <bound method Graph.__init__ of <networkx.classes.graph.Graph object at 0x108694438>>),
 ('__iter__',
  <bound method Graph.__iter__ of <networkx.classes.graph.Graph object at 0x108694438>>),
 ('__len__',
  <bound method Graph.__len__ of <networkx.classes.graph.Graph object at 0x108694438>>),
 ('__str__',
  <bound method Graph.__str__ of <networkx.classes.graph.Graph object at 0x108694438>>),
 ('add_cycle',
  <bound method Graph.add_cycle of <networkx.classes.graph.Graph object at 0x108694438>>),
 ('add_edge',
  <bound method Graph.add_edge of <networkx.classes.graph.Graph object at 0x108694438>>),
 ('add_edges_from',
  <bound method Graph.add_edges_from of <networkx.classes.graph.Graph object at 0x108694438>>),
 ('add_node',
  <bound method Graph.add_

In [13]:
from inspect import getmembers, ismethod
from copy import deepcopy
class udcBroker(object):
    def __init__(self, obj, function_filter=['__init__']):
        self.d = {}
        self.funcs = dict(getmembers(obj, ismethod))
        filtered_functions = {k: v for k, v in self.funcs.items() if k not in function_filter}
        self.d['obj'] = obj
        self.xbj = deepcopy(vars(obj))
        self.d.update(deepcopy(vars(obj)))  # somehow is enough
        self.d.update(filtered_functions)

        self.members_dict = self.d

In [15]:
udcBroker(G).xbj

{'node_dict_factory': dict,
 'adjlist_outer_dict_factory': dict,
 'adjlist_inner_dict_factory': dict,
 'edge_attr_dict_factory': dict,
 'graph': {},
 '_node': {0: {}, 1: {}, 2: {}},
 '_adj': {0: {2: {}}, 1: {2: {}}, 2: {0: {}, 1: {}}},
 'nodes': NodeView((0, 1, 2))}

In [14]:
udcBroker(G).funcs

{'__contains__': <bound method Graph.__contains__ of <networkx.classes.graph.Graph object at 0x108694438>>,
 '__getitem__': <bound method Graph.__getitem__ of <networkx.classes.graph.Graph object at 0x108694438>>,
 '__init__': <bound method Graph.__init__ of <networkx.classes.graph.Graph object at 0x108694438>>,
 '__iter__': <bound method Graph.__iter__ of <networkx.classes.graph.Graph object at 0x108694438>>,
 '__len__': <bound method Graph.__len__ of <networkx.classes.graph.Graph object at 0x108694438>>,
 '__str__': <bound method Graph.__str__ of <networkx.classes.graph.Graph object at 0x108694438>>,
 'add_cycle': <bound method Graph.add_cycle of <networkx.classes.graph.Graph object at 0x108694438>>,
 'add_edge': <bound method Graph.add_edge of <networkx.classes.graph.Graph object at 0x108694438>>,
 'add_edges_from': <bound method Graph.add_edges_from of <networkx.classes.graph.Graph object at 0x108694438>>,
 'add_node': <bound method Graph.add_node of <networkx.classes.graph.Graph o

In [10]:
g = UDO(udo=G)
g

{'methods': ['__contains__', '__getitem__', '__iter__', '__len__', '__str__', 'add_cycle', 'add_edge', 'add_edges_from', 'add_node', 'add_nodes_from', 'add_path', 'add_star', 'add_weighted_edges_from', 'adjacency', 'clear', 'copy', 'edge_subgraph', 'fresh_copy', 'get_edge_data', 'has_edge', 'has_node', 'is_directed', 'is_multigraph', 'nbunch_iter', 'neighbors', 'nodes_with_selfloops', 'number_of_edges', 'number_of_nodes', 'number_of_selfloops', 'order', 'remove_edge', 'remove_edges_from', 'remove_node', 'remove_nodes_from', 'selfloop_edges', 'size', 'subgraph', 'to_directed', 'to_directed_class', 'to_undirected', 'to_undirected_class', 'update'], 'node_dict_factory': <class 'dict'>, 'adjlist_outer_dict_factory': <class 'dict'>, 'adjlist_inner_dict_factory': <class 'dict'>, 'edge_attr_dict_factory': <class 'dict'>, 'graph': {}, '_node': {0: {}, 1: {}, 2: {}}, '_adj': {0: {2: {}}, 1: {2: {}}, 2: {0: {}, 1: {}}}, 'nodes': NodeView((0, 1, 2)), 'masked_members': ['obj']}

In [18]:
# udcBroker(G).get_members()
udcBroker(nx.barabasi_albert_graph(n, m)).members_dict

{'obj': <networkx.classes.graph.Graph at 0x111adbeb8>,
 'node_dict_factory': dict,
 'adjlist_outer_dict_factory': dict,
 'adjlist_inner_dict_factory': dict,
 'edge_attr_dict_factory': dict,
 'graph': {},
 '_node': {0: {}, 1: {}, 2: {}},
 '_adj': {0: {2: {}}, 1: {2: {}}, 2: {0: {}, 1: {}}},
 'nodes': NodeView((0, 1, 2)),
 '__contains__': <bound method Graph.__contains__ of <networkx.classes.graph.Graph object at 0x111adbeb8>>,
 '__getitem__': <bound method Graph.__getitem__ of <networkx.classes.graph.Graph object at 0x111adbeb8>>,
 '__iter__': <bound method Graph.__iter__ of <networkx.classes.graph.Graph object at 0x111adbeb8>>,
 '__len__': <bound method Graph.__len__ of <networkx.classes.graph.Graph object at 0x111adbeb8>>,
 '__str__': <bound method Graph.__str__ of <networkx.classes.graph.Graph object at 0x111adbeb8>>,
 'add_cycle': <bound method Graph.add_cycle of <networkx.classes.graph.Graph object at 0x111adbeb8>>,
 'add_edge': <bound method Graph.add_edge of <networkx.classes.gra

In [ ]:
balls = np.zeros(n,)

for node in G.nodes:
    rv = np.random.randint(1,25)
    G.nodes[node]['initial_balls'] = rv
    balls[node] = rv

In [ ]:
scale=100
nx.draw_kamada_kawai(G, node_size=balls*scale,labels=nx.get_node_attributes(G,'initial_balls'))

In [ ]:
initial_conditions = {'balls':balls, 'network':g}

In [ ]:
#input the deltas along the edges and update the boxes
#mechanism: edge by node dimensional operator
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 
# We make the state update functions less "intelligent",
# ie. they simply add the number of marbles specified in _input 
# (which, per the policy function definition, may be negative)


def update_balls(params, step, sL, s, _input):
    
    delta_balls = _input['delta']
    new_balls = s['balls']
    for e in G.edges:
        move_ball = delta_balls[e]
        src = e[0]
        dst = e[1]
        if (new_balls[src] >= move_ball) and (new_balls[dst] >= -move_ball):
            new_balls[src] = new_balls[src]-move_ball
            new_balls[dst] = new_balls[dst]+move_ball
            
    
    key = 'balls'
    value = new_balls
    
    return (key, value)

def update_network(params, step, sL, s, _input):
    
    new_nodes = _input['nodes']
    new_edges = _input['edges']
    new_balls = _input['quantity']
    
    graph = s['network']
    
    for node in new_nodes:
        graph.add_node(node)
        graph.nodes[node]['initial_balls'] = new_balls[node]
        graph.nodes[node]['strat'] = _input['node_strats'][node]
        
    for edge in new_edges:
        graph.add_edge(edge[0], edge[1])
        graph.edges[edge]['strat'] = _input['edge_strats'][edge]
    
    
    key = 'network'
    value = graph
    return (key, value)

def update_network_balls(params, step, sL, s, _input):
    
    new_nodes = _input['nodes']
    new_balls = _input['quantity']
    balls = np.zeros(len(s['balls'])+len(new_nodes))
    
    for node in s['network'].nodes:
        balls[node] = s['balls'][node]
    
    for node in new_nodes:
        balls[node] = new_balls[node]
            
    key = 'balls'
    value = balls
    
    return (key, value)

In [ ]:
# this time lets make three kinds of robots

def greedy_robot(src_balls, dst_balls):
    
    #robot wishes to accumlate balls at its source
    #takes half of its neighbors balls
    if src_balls < dst_balls:
        delta = -np.floor(dst_balls/2)
    else:
        delta = 0
    
    return delta

def fair_robot(src_balls, dst_balls):
    
    #robot follows the simple balancing rule
    delta = np.sign(src_balls-dst_balls)
    
    return delta


def giving_robot(src_balls, dst_balls):
    
    #robot wishes to gice away balls one at a time
    if src_balls > 0:
        delta = 1
    else:
        delta = 0
    
    return delta

In [ ]:
#in the previous version the robots were assigned to the edges
#moving towards an agent based model formulation we assign the stratgies
#instead to the nodes
robot_strategies = [greedy_robot,fair_robot, giving_robot]

for node in G.nodes:
    nstrats = len(robot_strategies)
    rv  = np.random.randint(0,nstrats)
    G.nodes[node]['strat'] = robot_strategies[rv]

for e in G.edges:
    owner_node = e[0]
    G.edges[e]['strat'] = G.nodes[owner_node]['strat']

In [ ]:
#Policy: node by edge dimensional operator
#input the states of the boxes output the deltas along the edges
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 
# We specify the robotic networks logic in a Policy Function
# unlike previous examples our policy controls a vector valued action, defined over the edges of our network
def robotic_network(params, step, sL, s):
    
    graph = s['network']
    
    pp.pprint(graph)
    
    
    delta_balls = {}
    for e in graph.edges:
        src = e[0]
        src_balls = s['balls'][src]
        dst = e[1]
        dst_balls = s['balls'][dst]
        
        #transfer balls according to specific robot strat
        strat = graph.edges[e]['strat']
        delta_balls[e] = strat(src_balls,dst_balls)
        
    return_dict = {'nodes': [],'edges': {}, 'quantity': {}, 'node_strats':{},'edge_strats':{},'delta': delta_balls}

    return(return_dict)

In [ ]:
def agent_arrival(params, step, sL, s):
    
    node= len(s['network'].nodes)
    edge_list = s['network'].edges
    
    #choose a m random edges without replacement
    #new = np.random.choose(edgelist,m) 
    new = [0, 1]#tester
    
    nodes = [node]
    edges = [(node,new_node) for new_node in new]
    
    initial_balls = {node:np.random.randint(1,25) }
    
    rv  = np.random.randint(0,nstrats)
    node_strat = {node: robot_strategies[rv]}
    
    edge_strats = {e: robot_strategies[rv] for e in edges}

    return_dict = {'nodes': nodes,
            'edges': edges, 
            'quantity':initial_balls, 
            'node_strats':node_strat,
            'edge_strats':edge_strats,
            'delta': np.zeros(node+1)
           } 
    return(return_dict)

In [ ]:
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 
# In the Partial State Update Blocks, the user specifies if state update functions will be run in series or in parallel
partial_state_update_blocks = [
    { 
        'policies': { # The following policy functions will be evaluated and their returns will be passed to the state update functions
            'p1': robotic_network
        },
        'variables': { # The following state variables will be updated simultaneously
            'balls': update_balls
            
        }
    },
    {
      'policies': { # The following policy functions will be evaluated and their returns will be passed to the state update functions
            'p1': agent_arrival
        },
        'variables': { # The following state variables will be updated simultaneously
            'network': update_network,
            'balls': update_network_balls
        }
    }
]

In [ ]:
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 
# Settings of general simulation parameters, unrelated to the system itself
# `T` is a range with the number of discrete units of time the simulation will run for;
# `N` is the number of times the simulation will be run (Monte Carlo runs)
# In this example, we'll run the simulation once (N=1) and its duration will be of 10 timesteps
# We'll cover the `M` key in a future article. For now, let's leave it empty
simulation_parameters = {
    'T': range(T),
    'N': 1,
    'M': {}
}

In [ ]:
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 
# The configurations above are then packaged into a `Configuration` object
config = Configuration(initial_state=initial_conditions, #dict containing variable names and initial values
                       partial_state_update_blocks=partial_state_update_blocks, #dict containing state update functions
                       sim_config=simulation_parameters #dict containing simulation parameters
                      )

In [ ]:
exec_mode = ExecutionMode()
exec_context = ExecutionContext(exec_mode.single_proc)
executor = Executor(exec_context, [config]) # Pass the configuration object inside an array
raw_result, tensor = executor.main() # The `main()` method returns a tuple; its first elements contains the raw results

In [ ]:
df = pd.DataFrame(raw_result)
balls_list = [b for b in df.balls]

In [ ]:
df

In [ ]:
plt.plot(df.timestep.values, balls_list)
plt.xlabel('iteration')
plt.ylabel('number of balls')
plt.title('balls in each box')
plt.legend(['Box #'+str(node) for node in range(n)], ncol = 2)

In [ ]:
end_state_balls = np.array([b for b in balls_list[-1]])
avg_balls = np.array([np.mean(b) for b in balls_list])

for node in G.nodes:
    G.nodes[node]['final_balls'] = end_state_balls[node]
    G.nodes[node]['avg_balls'] = avg_balls[node]

In [ ]:
cmap = plt.cm.jet
Nc = cmap.N
Ns = len(robot_strategies)
d = int(Nc/Ns)

k = len(G.edges)
strat_color = []
for e in G.edges:
    
    for i in range(Ns):
        if G.edges[e]['strat']==robot_strategies[i]:
            color = cmap(i*d)
            G.edges[e]['color'] = color
            strat_color = strat_color+[color]
            

In [ ]:
nx.draw_kamada_kawai(G, node_size=end_state_balls*scale, labels=nx.get_node_attributes(G,'final_balls'), edge_color=strat_color)
print(end_state_balls)

In [ ]:
rolling_avg_balls = np.zeros((T+1, n))
for t in range(T+1):
    for node in G.nodes:
        for tau in range(t):
            rolling_avg_balls[t,node] = (tau)/(tau+1)*rolling_avg_balls[t, node]+ 1/(tau+1)*balls_list[tau][node]

In [ ]:
plt.plot(range(len(rolling_avg_balls)),rolling_avg_balls)
plt.xlabel('iteration')
plt.ylabel('number of balls')
plt.title('time average balls in each box')
plt.legend(['Box #'+str(node) for node in range(n)], ncol = 2)

In [ ]:
for node in G.nodes:
    G.nodes[node]['avg_balls'] = int(10*(rolling_avg_balls[node][-1]))/10

nx.draw_kamada_kawai(G, node_size=avg_balls*scale, labels=nx.get_node_attributes(G,'avg_balls'))
print(end_state_balls)

In [ ]:
cmap = plt.cm.jet
Nc = cmap.N
Nt = len(simulation_parameters['T'])
dN = int(Nc/Nt)
cmaplist = [cmap(i*dN) for i in range(Nt)]

for t in simulation_parameters['T']:
    state = np.array([b for b in balls_list[t]])
    nx.draw_kamada_kawai(G, node_size=state*scale, alpha = .4/(t+1), node_color = cmaplist[t])